# Debugowanie i profilowanie kodu

## PDB
Python posiada zestaw narzędzi do debugowania, który jest wbudowany niemalże w każdą dystrybucję interpretera. Głównym jego elementem jest PDB, czyli po prostu Python Debugger. Aby uruchomić PDB można dodać po prostu następującą linijkę:

In [ ]:
breakpoint()

lub:

In [ ]:
import pdb
pdb.set_trace()

Czasami możemy chcieć zdebugować miejsce w kodzie, które rzuca wyjątkiem. Wówczas - jeśli jesteśmy w stanie wywołać błędną sytuację - możemy uruchomić debugowanie w trybie post mortem tak:

In [ ]:
import pdb
import crashing_module

crashing_module.crash()
pdb.pm()

Wówczas `pdb` skorzysta z informacji o tracebacku zapisanej w `sys.last_traceback` i otworzy sesję debugowania w stanie i miejscu po wystąpieniu wyjątku.

### Przydatne komendy w trybie debugowania:
- **b [miejsce]** - ustawia breakpoint w danym miejscu
- **c** - kontynuuje wykonanie aż do kolejnego breakpointu lub do końca programu
- **s** - wykonuje krok - albo wykonuje wywołanie funkcji wewnątrz której natychmiast staje albo przechodzi do kolejnej linii
- **n** - przechodzi do nowej linii w danej ramce
- **ll** - wypisuje aktualną linijkę kodu i jej otoczenie wewnątrz aktualnej ramki
- **u** - ramka w górę
- **d** - ramka w dół
- **w** - gdzie aktualnie znajduje się wykonanie
- **p [wyrażenie]** - wypisuje wartość wyrażenia
...
Wszystkie można znaleźć [tu](https://docs.python.org/3/library/pdb.html)


# cProfile

Czasami musimy ustalić, które kawałki kodu są wolne i z jakiego powodu. W tym celu przydają się profilery, z których niektóre znajdują się bezpośrednio w bibliotece standardowej Pythona. Dziś poznamy `cProfile`, który umożliwia:
- mierzenie czasu wykonania kodu
- mierzenie czasu wykonania kodu z rozbiciem na kroki
- sprawdzenie ile razy wykonały się poszczególne funkcje
- zrzucenie tych danych w formacie prostym do wizualizacji (np.: modułem `snakeviz`)

In [ ]:
import cProfile

code1 = """
lst = []
for i in range(10000):
    lst.append(i)
"""

code2 = """
lst = [i for i in range(10000)]
"""
cProfile.run(code1)
cProfile.run(code2)

Czasami chcemy mieć większą swobodę pracy z danymi zebranymi przez `cProfile` - wówczas możemy skorzystać z modułu `pstats`. Przykładowo, chcemy znaleźć 2 funkcje, które są wykonywane najwięcej razy. Robimy to tak:

In [5]:

def dodaj(l, el):
    l.append(el)

def sortuj(l):
    res = []
    for el in sorted(l):
        res.append(el)
    return res
    
def main():
    lst = [1,2,3]
    for i in range(1000):
        dodaj(lst, 2 * i)
    lst = sortuj(lst)

if __name__ == '__main__':
    import cProfile, pstats
    profiler = cProfile.Profile()
    profiler.enable()
    main()
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('ncalls')
    stats.print_stats(2)

         3007 function calls in 0.002 seconds

   Ordered by: call count
   List reduced from 6 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2003    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
     1000    0.000    0.000    0.001    0.000 /tmp/ipykernel_14863/2659033348.py:1(dodaj)




### Wizualizacja danych z cProfile
Aby zwizualizować dane zebrane przez cProfile wystarczy, że zainstalujemy bibliotekę `snakeviz`:
```
pip install snakeviz
```

Następnie jeśli chcemy jej użyć w Jupyterze musimy załadować rozszerzenie:
```
%load_ext snakeviz
```

i wywołać:
```
%snakeviz nazwa_funkcji()
```

In [7]:
# %pip install snakeviz
import random

def print_msg():
    for i in range(12):
        print("cześć!")


def generate_random_data():
    data = [random.randint(0, 99) for p in range(0, 1000)]
    return data

# Function to search 
def search_function(data):
    for i in data:
        if i in [111, 222, 333, 444, 555, 666]:
            print("znaleziono: ", i)

def main():
    data=generate_random_data()
    search_function(data)
    print_msg()


%load_ext snakeviz
%snakeviz main()

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
cześć!
 
*** Profile stats marshalled to file '/tmp/tmp1g5x0xwy'.
Embedding SnakeViz in this document...
<function display at 0x7f7eea9374c0>


## *Zadanie*

Spróbuj sprofilować dowolną funkcję z `watcher_service.py`. Co zajmuje najwięcej czasu? Jeśli napotykasz na problemy użyj `PDB` by poruszać się po wykonującym się kodzie i zdiagnozować ich przyczynę.